In [1]:
# TODO:
#   save weights for comparison groups with id names
#   allow plotting \ evaluation of weights
#   create tables for balance?
#   statistical tests for outcome variable - add s.e.
#   find a faster method to allow batch fitting (catboost? lightgbm?)

In [2]:
%cd /var/lib/projects

/var/lib/projects


In [3]:
import pandas as pd
import numpy as np
from provider_scorecard.benchmark import Benchmark

df = pd.read_csv("provider_scorecard/data/sparcs.zip")

/tmp/ipykernel_7944/2558641774.py:5: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("provider_scorecard/data/sparcs.zip")


In [4]:
def wt_att(y, score):
    """
    Calculate Average Treatment Effect on the Treated (ATT) weights.
    """
    weight = score / (1 - score)

    # define att weights
    wt = [1 if t == True else w for t, w in zip(y, weight)]

    return wt


def balance(var, X, y, wgt, digits=2):
    """
    Compute weighted balance statistics for ATT weights

    Weights are assumed to be 1 where y=True
    """

    tab = X[var]

    #
    tf, tc = tab[y], tab[~y]
    c_wgt = [v for v, c in zip(wgt, y) if not c]

    # compute average
    mf = round(np.average(tf), digits)
    mc = round(np.average(tc, weights=c_wgt), digits)

    return (mf, mc)

def evaluate(X,y,tr,wgt,digits=2):
    """
    Compute a difference-in-means estimate for the focal hospital
    """
    Xeval = X.copy()
    Xeval['tr'] = tr

    regr = LinearRegression()
    regr.fit(Xeval,y, sample_weight=wgt)

    return round(regr.coef_[len(Xeval.columns)-1],digits)

In [5]:
df['Hospital Service Area'].unique()

array(['New York City', nan, 'Hudson Valley', 'Central NY', 'Long Island',
       'Capital/Adirond', 'Finger Lakes', 'Western NY', 'Southern Tier'],
      dtype=object)

In [6]:
# filter on region
# get only the top 30 drg codes
df = df[df['Hospital Service Area'] == 'Central NY']

# some recoding
df['Length of Stay'] = [120 if x == '120 +' else x for x in df['Length of Stay']]

df['Length of Stay'] = df['Length of Stay'].astype(int)
df['APR DRG Code'] = df['APR DRG Code'].astype(str)

# top 30 drg codes
drg = df.groupby('APR DRG Code')['APR DRG Code'].count().sort_values(ascending=False).head(30)
drg = drg.astype(str)

In [38]:
df['Facility Name'].unique()

array(['Crouse Hospital - Commonwealth Division',
       'Auburn Community Hospital', 'Crouse Hospital',
       'Community Memorial Hospital Inc',
       'Guthrie Cortland Medical Center', 'River Hospital, Inc.',
       'Lewis County General Hospital', 'Gouverneur Hospital',
       'Faxton-St Lukes Healthcare St Lukes Division',
       'St Elizabeth Medical Center', 'Oneida Health Hospital',
       'Massena Hospital', 'Oswego Hospital',
       'Lakeview Center for Mental Health and Wellness',
       'Samaritan Medical Center',
       'University Hospital SUNY Health Science Center',
       'UPSTATE University Hospital at Community General',
       'Canton-Potsdam Hospital', 'Carthage Area Hospital Inc',
       'Cayuga Medical Center at Ithaca', 'Rome Memorial Hospital, Inc',
       'Claxton-Hepburn Medical Center', 'Little Falls Hospital',
       "St. Joseph's Hospital Health Center", 'Clifton-Fine Hospital'],
      dtype=object)

In [26]:
# input features
xfeat = [
    "Age Group",
    "Gender",
    "Race",
    "Length of Stay",
    "Type of Admission",
    "APR DRG Code",
    "APR Severity of Illness Description",
    "APR Risk of Mortality",
]

# evaluation features
efeat = ["Total Charges","Total Costs"]

# focal hospital
focal = "Auburn Community Hospital"

In [27]:
# create evaluation data
sub = df[df['APR DRG Code'].isin(drg.index)]

# model data
tr = sub['Facility Name'] == focal

X = sub[xfeat]
X = pd.get_dummies(X, drop_first=True, dtype=int)

In [28]:
benchmark_test = Benchmark(data = X, treat = tr)
benchmark_test.fit(lrate=0.1, nest=500)
benchmark_test.calc_balance()

In [31]:
for feat in efeat:
    res = benchmark_test.evaluate(y=sub[feat])
    print(f"{feat}:{res}")

Total Charges:-7252.44
Total Costs:-4253.26


In [ ]:
for x in sub['']

In [36]:
# let's test 5 hospitals
hosp_list = [
    "Auburn Community Hospital",
    "Crouse Hospital",
    "Community Memorial Hospital Inc",
    "Guthrie Cortland Medical Center",
]

outcome_list = []

for h in hosp_list:
    tr = sub['Facility Name'] == h

    bench = Benchmark(data = X, treat = tr)
    bench.fit(lrate=0.1,nest=500)
    
    out = bench.evaluate(sub['Total Costs'])

    outcome_list.append(out)

In [37]:
pd.DataFrame({'Hospital':hosp_list,'Total Costs':outcome_list})

,Hospital,Total Costs
0,Auburn Community Hospital,-4253.26
1,Crouse Hospital,-3518.27
2,Community Memorial Hospital Inc,-380.64
3,Guthrie Cortland Medical Center,-175.93
